In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
csv_path = "."
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
class CsvDataset(Dataset):
    def __init__(self, path):
        super(CsvDataset).__init__()
        csv = pd.read_csv(path)
        csv_np = csv.to_numpy()
        for i in [1, 3, 4, 5, 6, 7, 8]:
            wordset = {word: idx for idx, word in enumerate(np.unique(csv_np[:,i]))}
            for row in range(len(csv_np)):
                csv_np[row][i] = wordset[csv_np[row][i]]
        self.ints = torch.from_numpy(np.array(csv_np[:,[1,3,4,5,6,7,8]], dtype="int"))
        self.floats = torch.from_numpy(np.array(csv_np[:,[2,9,10,11]], dtype="float")).float()
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
        
        
    def __getitem__(self,idx):
        return self.ints[idx],self.floats[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.ints)

In [5]:
train_dataset = CsvDataset(csv_path+'/train.csv')
test_dataset = CsvDataset(csv_path+'/test.csv')
val_dataset = CsvDataset(csv_path+'/val.csv')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [6]:
class CsvNet(nn.Module):
    def __init__(self):
        super(CsvNet, self).__init__()
        self.emb1 = torch.nn.Embedding(8, 20)
        self.emb2 = torch.nn.Embedding(11, 20)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 20)
        self.emb5 = torch.nn.Embedding(4, 20)
        self.emb6 = torch.nn.Embedding(4, 20)
        self.emb7 = torch.nn.Embedding(7, 20)
        self.act = nn.ReLU()
        self.fc = nn.Linear(4, 80)
        self.bn = nn.BatchNorm1d(80)
        self.fc1 = nn.Linear(220, 8192)
        self.fc2 = nn.Linear(8192, 8192)
        self.fc3 = nn.Linear(8192, 4096)
        self.fc4 = nn.Linear(4096, 2048)
        self.fc5 = nn.Linear(2048, 1024)
        self.fc6 = nn.Linear(1024, 1)
        self.bn1 = nn.BatchNorm1d(8192)
        self.bn2 = nn.BatchNorm1d(8192)
        self.bn3 = nn.BatchNorm1d(4096)
        self.bn4 = nn.BatchNorm1d(2048)
        self.bn5 = nn.BatchNorm1d(1024)
        self.dropout = nn.Dropout()
    
    def forward(self, x, y):
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        y = self.bn(self.fc(y))
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        x = self.dropout(self.act(self.bn1(self.fc1(x))))
        x = self.dropout(self.act(self.bn2(self.fc2(x))))
        x = self.dropout(self.act(self.bn3(self.fc3(x))))
        x = self.dropout(self.act(self.bn4(self.fc4(x))))
        x = self.dropout(self.act(self.bn5(self.fc5(x))))
        return self.fc6(x)

In [7]:
model = CsvNet().to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for X1, X2, Y in train_loader:
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()
        prediction = model(X1, X2)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for X1_val, X2_val, Y_val in val_loader:
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(X1_val, X2_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 4372.2964, val_loss: 4660.3281
[Epoch:  2] Average loss: 4371.7129, val_loss: 4500.4321
[Epoch:  3] Average loss: 4418.8618, val_loss: 4502.1011
[Epoch:  4] Average loss: 4270.3027, val_loss: 4730.6484
[Epoch:  5] Average loss: 4384.2881, val_loss: 4167.6318
[Epoch:  6] Average loss: 4431.3789, val_loss: 4690.7949
[Epoch:  7] Average loss: 4343.6523, val_loss: 4170.4009
[Epoch:  8] Average loss: 4466.0366, val_loss: 4647.4604
[Epoch:  9] Average loss: 4412.6699, val_loss: 4586.2441
[Epoch: 10] Average loss: 4241.2085, val_loss: 4234.9453
[Epoch: 11] Average loss: 4344.7949, val_loss: 4580.3384
[Epoch: 12] Average loss: 4271.6592, val_loss: 4645.9526
[Epoch: 13] Average loss: 4277.5532, val_loss: 4147.1143
[Epoch: 14] Average loss: 4316.7012, val_loss: 4470.2388
[Epoch: 15] Average loss: 4259.7266, val_loss: 4054.2080
[Epoch: 16] Average loss: 4155.3804, val_loss: 4415.8770
[Epoch: 17] Average loss: 3988.6382, val_loss: 4100.4014
[Epoch: 18] Average loss: 4193.

[Epoch: 144] Average loss: 1281.4932, val_loss: 1442.6848
[Epoch: 145] Average loss: 1416.3638, val_loss: 1607.2837
[Epoch: 146] Average loss: 1280.4219, val_loss: 1786.6837
[Epoch: 147] Average loss: 1141.3365, val_loss: 1496.4762
[Epoch: 148] Average loss: 2176.0356, val_loss: 1863.8762
[Epoch: 149] Average loss: 1323.3811, val_loss: 1567.6130
[Epoch: 150] Average loss: 1252.4796, val_loss: 1711.6079
[Epoch: 151] Average loss: 1154.3741, val_loss: 1585.8553
[Epoch: 152] Average loss: 1136.2388, val_loss: 1461.4293
[Epoch: 153] Average loss: 1174.5972, val_loss: 1473.1067
[Epoch: 154] Average loss: 1132.6957, val_loss: 1627.9971
[Epoch: 155] Average loss: 1221.6715, val_loss: 1931.8945
[Epoch: 156] Average loss: 1078.2112, val_loss: 1628.9897
[Epoch: 157] Average loss: 1159.3375, val_loss: 1597.8983
[Epoch: 158] Average loss: 1221.7819, val_loss: 1504.3204
[Epoch: 159] Average loss: 1179.6843, val_loss: 1778.8434
[Epoch: 160] Average loss: 1217.4443, val_loss: 1600.9076
[Epoch: 161] A

In [9]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for X1_test, X2_test, Y_test in test_loader:
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X1_test, X2_test)
        mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 75.25%
